In [ ]:
# @title Import Library
import pandas as pd
import csv
from transformers import pipeline
from tqdm import tqdm

In [ ]:
# @title Extract Features using spaCy
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

df = pd.read_csv("/content/sample_data/linkedin_profile_process.csv")
df.head(4)


,no,name,title,link,snippet,Unnamed: 5,Unnamed: 6
0,1,Duc Quang Tran,Front End Developer Intern | Computer Science ...,https://www.linkedin.com/in/quangductran/,Front End Developer Intern | Computer Science ...,NaN,NaN
1,2,Huy Anh Nguyen,Frontend Developer - 1QBit | LinkedIn,https://ca.linkedin.com/in/huyanh149,"Since I started working at Base, I have been p...",NaN,NaN
2,3,Sahar Mirkamrani,Frontend Developer | LinkedIn,https://www.linkedin.com/in/sahar-mirkamrani,Frontend Developer Intern. MAPSA HR & Training...,NaN,NaN
3,4,Julia Kuznetsova,Web Developer Intern @BestLogic Staffing ...,https://www.linkedin.com/in/juliakuznetsovaqa,Web Developer Intern @BestLogic Staffing | Fro...,NaN,NaN


In [ ]:
import numpy as np
import re
import string

# Skill normalization mapping
skill_normalization_map = {
    "c++": "cpp",
    "c#": "csharp",
    "bash/shell": "bash shell",
    "html/css": "html css",
}

def clean_text(text):
    """Clean text by removing unnecessary punctuation and converting to lowercase"""
    if pd.isna(text) or text == '':
        return ''

    text = str(text)

    # Convert to lowercase
    text = text.lower()
    text = re.sub(r'["""''`]', '', text)  # Remove quotes
    text = re.sub(r'[(){}\[\]]', '', text)  # Remove brackets
    text = re.sub(r'[!@#$%^&*+=|\\:;"<>?]', '', text)  # Remove special chars
    text = re.sub(r'[,]{2,}', ',', text)  # Multiple commas to single
    text = re.sub(r'\s+', ' ', text)  # Multiple spaces to single

    # Apply skill normalization
    for old_skill, new_skill in skill_normalization_map.items():
        text = re.sub(r'\b' + re.escape(old_skill) + r'\b', new_skill, text)

    return text.strip()

# Process title and snippet columns
df['title'] = df['title'].apply(clean_text)
df['snippet'] = df['snippet'].apply(clean_text)

# Save processed data
df.to_csv("processed_linkedin_data.csv", index=False)

print("\nProcessing completed!")
print("Saved processed data to: processed_linkedin_data.csv")


Processing completed!
Saved processed data to: processed_linkedin_data.csv


In [ ]:
df.head(4)

,no,name,title,link,snippet,Unnamed: 5,Unnamed: 6
0,1,Duc Quang Tran,front end developer intern computer science ...,https://www.linkedin.com/in/quangductran/,front end developer intern computer science gr...,NaN,NaN
1,2,Huy Anh Nguyen,frontend developer - 1qbit linkedin,https://ca.linkedin.com/in/huyanh149,"since i started working at base, i have been p...",NaN,NaN
2,3,Sahar Mirkamrani,frontend developer linkedin,https://www.linkedin.com/in/sahar-mirkamrani,frontend developer intern. mapsa hr training. ...,NaN,NaN
3,4,Julia Kuznetsova,web developer intern bestlogic staffing ...,https://www.linkedin.com/in/juliakuznetsovaqa,web developer intern bestlogic staffing fronte...,NaN,NaN


In [ ]:
sentences = df["snippet"].dropna().tolist()

cleaned_sentences = []
for sentence in sentences:
    doc = nlp(sentence)
    filtered_tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    cleaned_sentences.append(filtered_tokens)

df["cleaned_tokens"] = cleaned_sentences

df.to_csv("cleaned_output.csv", index=False)

print(df.head())

   no              name                                            title  \
0   1    Duc Quang Tran  front end developer intern computer science ...   
1   2    Huy Anh Nguyen              frontend developer - 1qbit linkedin   
2   3  Sahar Mirkamrani                      frontend developer linkedin   
3   4  Julia Kuznetsova      web developer intern bestlogic staffing ...   
4   5     Harsh Thakran     ex internandroid development azureskynet ...   

                                            link  \
0      https://www.linkedin.com/in/quangductran/   
1           https://ca.linkedin.com/in/huyanh149   
2   https://www.linkedin.com/in/sahar-mirkamrani   
3  https://www.linkedin.com/in/juliakuznetsovaqa   
4       https://in.linkedin.com/in/harsh-thakran   

                                             snippet  Unnamed: 5  Unnamed: 6  \
0  front end developer intern computer science gr...         NaN         NaN   
1  since i started working at base, i have been p...         NaN      